# Prediction Engine

In [1]:
# importing libraries
import pandas as pd
import numpy as np
from sklearn import linear_model 

In [2]:
# reading clean file
file_path = "C:/Github Repo/NBA-Analytics/data/basketballProject/enriched-files/enriched-data.csv"
enriched_df = pd.read_csv(file_path)
# remove current year data to create training data
training_df = enriched_df[(enriched_df["SEASON_ID"] != "2022-23") & (1 == 1)]
# create test data from current year
test_df = enriched_df[(enriched_df["SEASON_ID"] == "2022-23") & (1 == 1)].reset_index()
test_df["PTS_PRED"] = 0

In [3]:
# create x and y linear regression model
x = training_df[["AGE","SEASON_NO","MIN"]]
y = training_df["PTS"]

regr = linear_model.LinearRegression()
regr.fit(x.values,y.values)

# create df to feed into prediction
predict_df = test_df[["AGE","SEASON_NO","GP","MIN"]]

# create variables to scale games/minutes to a full season
min_test_max = max(test_df["MIN"])
min_his_max = max(predict_df["MIN"])

for index, row in test_df.iterrows():
    name = row["PLAYER"]
    age = row["AGE"]
    season = row["SEASON_NO"]
    # formula for deriving minutes: MIN/MIN_TEST_MAX = MIN_W/MIN_HIS_MAX
    minutes = (row["MIN"]*min_his_max)/min_test_max
    pts_pred = regr.predict([[age,season,minutes]])[0]
    test_df.loc[(test_df["PLAYER"] == name) & (1 == 1), "PTS_PRED"] = pts_pred

In [4]:
test_df["PRED_DIFF"] = test_df["PTS"] - test_df["PTS_PRED"]

In [5]:
file_path = "C:/Github Repo/NBA-Analytics/data/basketballProject/prediction-files/"
# file_path = "../../data/basketballProject/prediction-files/"
file_name = "predicted-data"
file_extension = ".csv"
full_path = file_path + file_name + file_extension
test_df.to_csv(full_path)

In [6]:
# test_df[["PLAYER","PTS","PTS_PRED","PRED_DIFF"]][test_df["PRED_DIFF"] > 300]